# Cell state dynamics of skin squamous cell carcinoma dependent on colocalized cell population
Exdyn can estimate cell state dynamics dependent on multivariate extrinsic factors such as colocalization with neighboring cells and chromatin accessibility. Exdyn provide several downstream analysis for exploring the contributing fetures of the extrinsic factors. Here, we estimate cell state dynamics of human skin squamous cell carcinoma dependen on the colocalization scores with cells other than cancer cells, which were estimated by DeepCOLOR (). 

## Model setup and optimization
Exdyn accepts AnnData object for dynamics estimation. The model parameters such as the number of neural network layers can be specified through a dictionary of the parameters.

In [ ]:
import scanpy as sc
from exdyn import workflow, modules, visualization
import numpy as np
import torch
import random
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


adata_file_path = 'data/scc/tumor_adata.h5ad'
coloc_adata_file_path = 'data/scc/coloc_adata.h5ad'
lt_df_file_path = 'data/scc/lt_df.csv'
adata = sc.read_h5ad(adata_file_path)
top_corr_genes, gene_names = workflow.extract_top_corr_genes(adata)
adata.var['dynamics_genes'] = False
adata.var['dynamics_genes'][top_corr_genes] = True

model_params = {
            'x_dim': 100,
            'z_dim': 10,
            'enc_z_h_dim': 128, 'enc_d_h_dim': 128, 'dec_z_h_dim': 128,
            'num_enc_z_layers': 2, 'num_enc_d_layers': 2,
            'num_dec_z_layers': 2, 'use_vamp': False, 'loss_mode': 'nb'
}
checkpoint_dirname = 'checkpoint'
adata.obs['sample'] = 'Sample'
est_adata, model = workflow.conduct_cvicdyf_inference(adata, model_params, checkpoint_dirname, batch_size=128, two_step=False, dyn_mode=False, epoch=1000, patience=50, module=modules.Cvicdyf, use_highly_variable=False, batch_key='sample', condition_key='coloc', cond_in_obsm=True)

## Visualize purturbation of cell state dynamics by extrinsic factors
Exdyn can estimate the purturbation effect on cell state dynamics by extrinsic factors through singular value decompotion of Jacobian matrix.
Firstly, you can visualize the cell state dynamics given original extrinsic factors.

In [ ]:
est_adata.obs['Annotation'] = est_adata.obs['level3_celltype']
visualization.plot_mean_flow(est_adata, cluster_key='Annotation', legend_loc='on data')

Next, you can conduct SVD of Jacobian matrix and identify the pricipal purtubations and the effects on cell state dynamics. Here, you can visualize the purtubed dynamics for top 3 modes.

In [ ]:
top_modes = 4
est_adata = workflow.conduct_jacobian_analysis(est_adata, model, top_modes=top_modes)
from matplotlib import pyplot as plt
fig, axes = plt.subplots(1, top_modes, figsize=(6 * top_modes, 5 * 1))
for mode in range(top_modes):
   ax = axes[mode] 
   visualization.diff_visualize(est_adata.obsm[f'dd_dc_embed{mode}'], est_adata.obsm['X_umap'], ax)
   ax.set_title(f"Dynamics difference of mode {mode + 1}")

fig.show()

We note that the contribution of each feature of extrinsic factors to each mode of principal purturbation can be found in `adata.uns['dd_dc_v']`. This can be useful for identify the fetures which can cause specific cell state transitions.

In [ ]:
est_adata.uns['dd_dc_v']

You can also quantify the induction of each cell cluster by each purturbation.

In [ ]:
mode = 3
sign = 1
cluster = 'TSK'
est_adata = workflow.analyze_flux_into_clusters(est_adata, cluster, mode, sign, 'Annotation')
sc.pl.umap(est_adata, color=['Annotation', f'mode{mode * sign}_flux_into_{cluster}'])

## Analyze cell cell interacraction
In the case where the extrinsic factors are colozalizaiton scores with neighboring cells, you can analyze cell cell interaction which mediate specific mode of principal purturbation patterns of cell state dynamics. Firstly, you can annotate and visualize the contribution of neighboring cells to each mode of the purturbations.

In [ ]:
coloc_adata = sc.read_h5ad(coloc_adata_file_path)
cov_labels = []
for i in range(top_modes):
    cov_label = f'coloc_cov_mode{i}'
    coloc_adata.obs[cov_label] = est_adata.uns['dd_dc_v'][:, i]
    cov_labels.append(cov_label)

sc.pl.umap(coloc_adata, color=cov_labels + ['level2_celltype'], legend_loc='on data')

Here, we extract cancer cells directed TSK by mode 1 perturbation and quantify the activity of each ligand for the perturbation.

In [ ]:
import pandas as pd
import numpy as np
top_tsk_adata = est_adata[est_adata.obs[f'mode{mode * sign}_flux_into_{cluster}'] > np.quantile(est_adata.obs[f'mode{mode * sign}_flux_into_{cluster}'], 0.95)]
top_tsk_adata = workflow.calc_covdiff(top_tsk_adata, model, mode, sign, 'coloc', 'sample', sep_num=5)
condiff_vec = pd.Series(top_tsk_adata.layers[f'cov_diff_{sign * mode}'].mean(axis=0), top_tsk_adata.var_names)
lt_df = pd.read_csv(lt_df_file_path, index_col=0)
ligacs = workflow.analyze_ligand_activity(condiff_vec, lt_df)


In order to derive plausible candidates for cell cell interaction, you can intersects genes with high ligand activity with positive DEGs for contributing neighboring cells.

In [ ]:
coloc_adata.obs['target_coloc'] = (coloc_adata.obs['coloc_cov_mode1'] > np.quantile(coloc_adata.obs['coloc_cov_mode1'], 0.9)).astype(str)
sc.tl.rank_genes_groups(coloc_adata, 'target_coloc', groups=['True'], reference='False', method='wilcoxon')
deg_df = sc.get.rank_genes_groups_df(coloc_adata, 'True')
pos_degs = deg_df.query('pvals_adj < 0.05 and logfoldchanges > 1').names

ligacs[np.intersect1d(pos_degs, ligacs.index)].sort_values(ascending=False)
